In [5]:
# Cell 1 — warm-up & controller check (no shutdown)
import os, sys, importlib
import rclpy
from rclpy.node import Node

print("Python:", sys.version.split()[0], "ROS_DISTRO:", os.environ.get("ROS_DISTRO"))

TOPICS = [
    "/ros_robot_controller/set_motor_speeds",
    "/ros_robot_controller/enable_reception",
]

def check_controller():
    if not rclpy.ok():
        rclpy.init(args=None)
    n = Node("rm_health_check")
    try:
        for t in TOPICS:
            subs = n.get_subscriptions_info_by_topic(t)
            pubs = n.get_publishers_info_by_topic(t)
            print(f"{t}  subs={len(subs)}  pubs={len(pubs)}")
        if len(n.get_subscriptions_info_by_topic(TOPICS[0])) == 0:
            print("⚠️ Controller likely not running. Start it:")
            print("   ros2 launch ros_robot_controller ros_robot_controller.launch.py")
    finally:
        n.destroy_node()
        # NOTE: do NOT call rclpy.shutdown(); we want the context alive for robot_moves

check_controller()

# Load robot_moves and set a smooth rate
import robot_controller_api as rca
import robot_moves as rm
importlib.reload(rca)
importlib.reload(rm)
try:
    rm.set_rate(40)
except Exception:
    pass

print("robot_moves loaded.")


Python: 3.10.12 ROS_DISTRO: humble
/ros_robot_controller/set_motor_speeds  subs=1  pubs=1
/ros_robot_controller/enable_reception  subs=1  pubs=1
robot_moves loaded.


In [6]:
# Cell 2 — kids' one-liners (seconds only) using robot_moves calibration
import time, importlib
import robot_moves as rm
importlib.reload(rm)

DEFAULT_SPEED = 200.0   # fixed for the class
AUTO_GAP      = 0.25    # small pause after each move

def _pause():
    if AUTO_GAP and AUTO_GAP > 0:
        time.sleep(AUTO_GAP)

# existing one-liners (keep for continuity)
def forward(seconds=0.7):    rm.forward(DEFAULT_SPEED, seconds); _pause()
def back(seconds=0.7):       rm.backward(DEFAULT_SPEED, seconds); _pause()
def left(seconds=0.7):       rm.left(DEFAULT_SPEED, seconds); _pause()
def right(seconds=0.7):      rm.right(DEFAULT_SPEED, seconds); _pause()
def turnleft(seconds=0.6):   rm.turn_left(DEFAULT_SPEED, seconds); _pause()
def turnright(seconds=0.6):  rm.turn_right(DEFAULT_SPEED, seconds); _pause()
def stop():                  rm.stop()
def horn():                  rm.horn()

# new: lower-case helpers that wrap the smooth /cmd_vel motions
def diagonal_left(seconds=0.8):   rm.diagonal_left(seconds); _pause()
def diagonal_right(seconds=0.8):  rm.diagonal_right(seconds); _pause()
def drift_left(seconds=1.2):      rm.drift_left(seconds); _pause()
def drift_right(seconds=1.2):     rm.drift_right(seconds); _pause()

print("Ready: Forward, Back, Left, Right, TurnLeft, TurnRight, Stop, Horn")
print("Also: diagonal_left, diagonal_right, drift_left, drift_right")



Ready: Forward, Back, Left, Right, TurnLeft, TurnRight, Stop, Horn
Also: diagonal_left, diagonal_right, drift_left, drift_right


In [7]:
# Cell 3 ? diagonal demo (45° moves)
print("Diagonal: L, R, L, R")
diagonal_left(0.8)
diagonal_right(0.8)
diagonal_left(1.0)
diagonal_right(1.0)
horn()

Diagonal: L, R, L, R


In [8]:
# Cell 4 — drift demo (curved paths)
print("Drift: left then right")
drift_left(8)
drift_right(8)

print("Figure-8 style (two mirrored drifts)")
drift_left(8)
drift_right(8)

horn()

Drift: left then right
Figure-8 style (two mirrored drifts)


In [4]:
stop()

In [5]:
forward(1)

In [6]:
back(1)